# Assigment 2 - Python dictionaries and file I/O

### Overview
------------

The GTEx gene expression dataset consists of samples taken from many human subjects across a variety of tissues. From this dataset we have identified a set of highly-expressed tissue-specific genes. In this exercise, you will be pulling the expressing data for those genes but only in their primary tissue of expression and investigating how much variability exists across the sample population.

Biological Learning Objectives

- Use metadata to associate data with subject or sample traits
- Use descriptive statistics to make inferences about gene expression heterogeneity

Computational Learning Objectives

- Create and modify a dicitionary
- Use keys to retrieve values stored in a dictionary
- Step through each key and value in a dictionary inside a `for` loop
- Open and read a text file
- Write to a text file

### Instructions
----------------

- Save a copy of this notebook as `~/qbXX-answers/python_dictionaries.ipynb`.
- Fill in answers in the available code/markdown cells below.
- Remember to comment your code to help yourself and us know what each part is intended to do.

In [1]:
# Use this dictionary for identifying genes of interest and their corresponding tissue of interest

gene_tissue = {
    'ENSG00000042832.11': 'Thyroid',
    'ENSG00000091704.9': 'Pancreas',
    'ENSG00000118245.2': 'Testis',
    'ENSG00000122304.10': 'Testis',
    'ENSG00000122852.14': 'Lung',
    'ENSG00000132693.12': 'Liver',
    'ENSG00000134812.7': 'Stomach',
    'ENSG00000135346.8': 'Pituitary',
    'ENSG00000137392.9': 'Pancreas',
    'ENSG00000142515.14': 'Prostate',
    'ENSG00000142615.7': 'Pancreas',
    'ENSG00000142789.19': 'Pancreas',
    'ENSG00000158874.11': 'Liver',
    'ENSG00000162438.11': 'Pancreas',
    'ENSG00000164816.7': 'Small Intestine - Terminal Ileum',
    'ENSG00000164822.4': 'Small Intestine - Terminal Ileum',
    'ENSG00000168925.10': 'Pancreas',
    'ENSG00000168928.12': 'Pancreas',
    'ENSG00000170890.13': 'Pancreas',
    'ENSG00000171195.10': 'Minor Salivary Gland',
    'ENSG00000172179.11': 'Pituitary',
    'ENSG00000175535.6': 'Pancreas',
    'ENSG00000175646.3': 'Testis',
    'ENSG00000182333.14': 'Stomach',
    'ENSG00000187021.14': 'Pancreas',
    'ENSG00000203784.2': 'Testis',
    'ENSG00000204983.13': 'Pancreas',
    'ENSG00000219073.7': 'Pancreas',
    'ENSG00000229859.9': 'Stomach',
    'ENSG00000240338.5': 'Pancreas',
    'ENSG00000254647.6': 'Pancreas',
    'ENSG00000256713.7': 'Stomach',
    'ENSG00000259384.6': 'Pituitary',
    }

1. Load sample metadata from "~/Data/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt" and create a dictionary of sample metadata associating each sample ID with the tissue it was sampled from

    - This file is tab-separated but some values have spaces in their text so make sure to specify `\t` in the `.split()` method
    - There are several columns in the metadata file, but you only need to be concerned with `SAMPID` and `SMTSD`, the sample ID and sample tissue, respectively
    - Your dictionary should use the `SAMPID` as the key and `SMTSD` as the value

In [6]:
# Create a dictionary to hold the metadata
metadata_fname = "/Users/cmdb/Data/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt"
fs = open(metadata_fname)
meta_dictionary = {}
# Step through each line of the metadata file
header = fs.readline() #.readline read untils the first readline character

# Remove the newline character from the line you just read in and split it by the tab character
for line in fs:
   line = line.strip("\n")
   line = line.split("\t")
   meta_dictionary[line[0]] = line[6] #equal sign pairs the values together, x = 0 (normal), dictionary key = value
    # Add the data to your metadata dictionary, using the "SAMPID" column value as the dictionary key and the "SMTSD" column value as the dictionary value

2. Load and process the header from the GTEx gene expression file "~/Data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

    - You have code that will open the file and read in the header line
    - Create a list of tissues corresponding to the column names in the header, one tissue per column name (don't worry about which the column is a sample ID or not since column names that aren't sample IDs will just get a "missing" label)
    - For each value in the header, see if it appears in the dictionary of sample ID/tissues that you created in step 1
    - If the value is in your dictionary, add the corresponding tissue name (the value in dictionary for that sample ID key) to your tissue list
    - If the value is not in your dictionary, add "missing" to the tissue list

In [13]:
# Because this file is a gzip-compressed file, you will be using the `gzip` library that comes with python
import gzip

expression_fname = "/Users/cmdb/Data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

# Open the file using the gzip library
fs = gzip.open(expression_fname)

# The file starts with two lines of information before the header, so these are skipped using 2 calls of `.readline()` without keeping the returned data
_ = fs.readline()
_ = fs.readline()

# Gzipped text files are read in as byte strings, not regular strings, so you will see that `.decode()` is included after `.readline()` to convert the input into a string
header = fs.readline().decode().rstrip().split("\t")

# Make sure to close the file when you are done reading it
fs.close()

# Create a list to hold the tissue names
tissues = []

# Step through each value in the header
#compare stuff in the header to the dictionary
for value in header:
   if value in meta_dictionary: #the thing i am lookihng for is frist and the thing im searching thru is second
      tissues.append(meta_dictionary[(value)])
   else: 
      tissues.append("missing")
    # Check if the value has an entry in the gene_tissue dictionary, adding either the tissue corresponding to that entry or "missing" to your tissues list


3. Create a list of expression values for each gene in the `gene_tissue` dictionary (given at the start of the exercise), keeping only expression values from the tissue of interest for that gene

    - Since the file is already open, you can keep reading lines from it in, picking up from after the last line read
    - You will be log-transforming the expression data using the `log2` function from the built-in library `math`
    - To deal with zeros in expression data, add one before log-transforming expression data

In [8]:
# Import the log2 function from the math library to transform the expression counts
from math import log2

# Create a dictionary to hold each gene's expression data
gene_expression = {}

# Because this is a large file, for testing purposes you can use only a small portion of the file to test your code
line_counter = 0

# Step through each line of the file
for line in gzip.open(expression_fname):

    # Convert the line into a string from a byte string, remove the newline character at the end, and split it by tab characters
    fields = line.decode().rstrip().split("\t")

    # Check if this line contains data for a gene we are interested in. If not, skip it
    fields = line.decode().rstrip().split("\t")
    if fields[0] not in gene_tissue:
        continue

    # Determine which tissue we care about for this gene
    tissue = gene_tissue[fields[0]]

    # Create a list in the gene expression dictionary to hold the gene's data
    gene_expression[fields[0]] = []

    # Look at each column position, one by one
    for i in range(len(tissues)):

        # For each column, see if it is from the tissue we care about (using the tissues list). If so, add that field to our gene expression list, first transforming it with log2(1 + float(field[i])) 
        if tissues[i] == tissue:
            gene_expression[fields[0]].append(log2(1 + float(fields[i])))

4. For each gene, calculate the mean, standard deviation, and relative standard deviation and write them to a results file

    - You will be using the `mean` and `stdev` functions from the built-in `statistics` library. Both functions accept a list as their input
    - Format your results for each gene into a string and write that string to your results file
    - If you want to organize your results better (completely optional!!), you can save your results in a list and sorting them by tissue type before writing them to a file

In [46]:
# Load the functions to caculcate the mean and standard deviation from a list
from statistics import stdev, mean

# Open a file to write the results to
output = open("result_file.txt", "w")
tis_list = []
# Step through each gene and its expression value list
for gene in gene_expression:
    expression_value_list = gene_expression.get(gene)
    # Calculate the mean and standard deviation of expression
    gene_mean = mean(expression_value_list)
    gene_SD = stdev(expression_value_list)
    # Calculate the relative standard deviation (coefficient of variation, std / mean)
    gene_coefficient_of_variation = gene_SD/gene_mean
    tis = gene_tissue[gene] 
    # Write the tissue namne, gene name, mean, standard deviation, and relative standard deviation to your results file
    #Close
    print(f"Tissue name: {tis}, Gene: {gene}, Gene mean: {gene_mean}, Gene standard deviation: {gene_SD}, Gene coefficient of variation: {gene_coefficient_of_variation}, \n")
    output.write(f"Tissue name: {tis}, Gene: {gene}, Gene mean: {gene_mean}, Gene standard deviation: {gene_SD}, Gene coefficient of variation: {gene_coefficient_of_variation}, \n")
    tissue_list = []
    sorted(tis)
    tissue_list.append(tis)
    tissue_list.append(gene_coefficient_of_variation)
    tis_list.append(tissue_list)
    tis_list.sort()
for sort in tis_list:
    print(sort)
output.close()

Tissue name: Pancreas, Gene: ENSG00000162438.11, Gene mean: 11.64813927955005, Gene standard deviation: 0.757491204101109, Gene coefficient of variation: 0.06503109088255767, 

Tissue name: Pancreas, Gene: ENSG00000142615.7, Gene mean: 11.982260919355417, Gene standard deviation: 1.4113620914457623, Gene coefficient of variation: 0.1177876279731093, 

Tissue name: Pancreas, Gene: ENSG00000219073.7, Gene mean: 13.186023575042132, Gene standard deviation: 0.6691398656550582, Gene coefficient of variation: 0.05074614510181628, 

Tissue name: Pancreas, Gene: ENSG00000142789.19, Gene mean: 14.651792140427176, Gene standard deviation: 0.5222110990262671, Gene coefficient of variation: 0.03564144877440514, 

Tissue name: Testis, Gene: ENSG00000203784.2, Gene mean: 10.588661238478307, Gene standard deviation: 1.2485963275387792, Gene coefficient of variation: 0.11791824286544222, 

Tissue name: Liver, Gene: ENSG00000132693.12, Gene mean: 11.233820059075176, Gene standard deviation: 3.230343826

5. Looking at your results, answer the following questions

    - Which tissues have low relative variability (rv < 0.2), medium relative variability (0.2 < rv < 0.4), or high relative variability (rv > 0.4)?
    - Do each of the genes for a given tissue show the same level of relative variability?
    - Propose an explanation for differences in relative variability you see between tissues based on what you know about their functions.
    - Were any of the results surpising to you?

# (Calculations shown below)

# Part 1
The liver, pancreas, pituitary, testies, and thyroid have low average relative variability. The lung, minor salivary gland, prostate, and small intestine have medium average relative variability. The stomach is the only tissue with high average relative variability.

# Part 2
No, the genes for a given tissue is highly variable. For example, the liver, although it has a low sample size, has different level of variability with one of the expression levels for the sample at ~0.05 while the other liver sample has an expression level at ~0.29. On the other hand, the testies and pancreas are tissues that seem to have the same level of relative variability at ~ 0.10 and ~0.04, respectively.

# Part 3
A possible reason for the difference in relative variability seen in the tissue might be due to the rate of cell regeneration or specific enzymes that encode for digestion. For example, it seems that the relative variability for the stomach and salivary gland are high which are correlated to digestion. In a similar vein, these cells regnerate at a higher rate than the ograns listed with a lower variability. Therefore, there may be these two possible reasons for the results seen.

# Part 4
I am surprised by the low variability in the pancreas given how important its role is in glucose uptake and dyresgulation correlating with type I diabetes.

In [ ]:
#Calculations
liver = (0.054821389153249356 + 0.2875552403125156)/2
lung = (0.2873173022007689)
minor_salivary_gland = (0.2568165163261053)
pancreas = (0.02959391720045206 + 0.03564144877440514 + 0.04030574392803288 +0.04064669060038226 + 0.04640583295252949 + 0.049139407233868164 + 0.05074614510181628 + 0.05356319299548535 + 0.05552296526199176 + 0.06028688906210789 + 0.06503109088255767 + 0.10821010189920609 + 0.1177876279731093 + 0.17138691316950458)/14
pituitary = (0.13183533543166376 + 0.13927854797333378 + 0.18301718980763804)/3
prostate = 0.24494933157655346
small_intestine = (0.2611958402727668 + 0.30069395169127305)/2
stomach = (0.4007168440566386 + 0.4763541021105254 + 0.5008484862295736 + 0.5262730937685776)/4
testies = (0.10195561883308456 + 0.10761864220781056 + 0.11335847046308609 + 0.11791824286544222)/4
thyroid = 0.07463416023060992

print(f"Liver rv avg: {liver}")
print(f"Lung rv avg: {lung}")
print(f"Minor salivary gland rv avg: {minor_salivary_gland}")
print(f"Pancreas rv avg: {pancreas}")
print(f"Pituitary rv avg: {pituitary}")
print(f"Prostate rv avg: {prostate}")
print(f"Small intestine rv avg: {small_intestine}")
print(f"Stomach rv avg: {stomach}")
print(f"Testies rv avg: {testies}")
print(f"Thyroid rv avg: {thyroid}")

Liver rv avg: 0.17118831473288249
Lung rv avg: 0.2873173022007689
Minor salivary gland rv avg: 0.2568165163261053
Pancreas rv avg: 0.06601914050253206
Pituitary rv avg: 0.15137702440421186
Prostate rv avg: 0.24494933157655346
Small intestine rv avg: 0.28094489598201994
Stomach rv avg: 0.47604813154132875
Testies rv avg: 0.11021274359235585
Thyroid rv avg: 0.07463416023060992
